# 타이타닉

최규빈  
2023-09-26

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-z8ceMijNQrcczhH3ocuP1Y&si=UehL2BELwmid586_>

# 2. imports

In [52]:
import numpy as np 
import pandas as pd 
import sklearn.impute
import sklearn.linear_model
import sklearn.ensemble
import missingno as msno

# 3. 자료불러오기

`-` 아래의 코드는 `kaggle.json` 파일이 셋팅된 codespace 혹은
`kaggle.json` 파일이 셋팅된 리눅스 컴퓨터가 있는 경우만 사용가능 (이런거
셋팅하는 방법은 대면수업에서..)

In [53]:
!kaggle competitions download -c titanic
!unzip titanic.zip -d ./titanic
df_train = pd.read_csv('titanic/train.csv')
df_test = pd.read_csv('titanic/test.csv')
submission = pd.read_csv('titanic/gender_submission.csv')
!rm titanic.zip
!rm -rf titanic/

  0%|                                               | 0.00/34.1k [00:00<?, ?B/s]
100%|███████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 406kB/s]
Archive:  titanic.zip
  inflating: ./titanic/gender_submission.csv  
  inflating: ./titanic/test.csv      
  inflating: ./titanic/train.csv     

`-` 리눅스서버를 사용할 수 없는 경우 download 후 압축풀어서 사용

# 4. 결측치 확인 및 처리

## A. 결측치 체크

`-` 결측치확인

In [54]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB

## C. 결측치 처리

In [55]:
def impute_missing(df):
    df_imputed = df.copy()
    df_num = df.select_dtypes(include="number")
    df_cat = df.select_dtypes(exclude="number")
    df_imputed[df_num.columns] = sklearn.impute.SimpleImputer().fit_transform(df_num) 
    df_imputed[df_cat.columns] = sklearn.impute.SimpleImputer(strategy='most_frequent').fit_transform(df_cat) 
    return df_imputed

# 5. 분석

## A. 자료의 정리

In [56]:
df_train = df_train.eval('Fsize = SibSp + Parch + 1')
df_test = df_test.eval('Fsize = SibSp + Parch + 1')

In [57]:
f = lambda x,y: str(x)+str(y) if not (pd.isna(x) or pd.isna(y)) else np.nan

In [58]:
df_train = df_train.assign(Pclass_Embarked = list(map(f, df_train.Pclass,df_train.Embarked)))
df_test= df_test.assign(Pclass_Embarked = list(map(f, df_test.Pclass,df_test.Embarked)))

In [59]:
df_train.Pclass_Embarked.unique()

In [75]:
features = ['Pclass','Sex','SibSp','Parch','Pclass_Embarked']
#features = ['Sex','Age','Fare']
X = pd.get_dummies(impute_missing(df_train[features]))
y = impute_missing(df_train)[['Survived']]
XX = pd.get_dummies(impute_missing(df_test[features]))

## B. predictor 생성

In [76]:
predictr = sklearn.ensemble.RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

## C. 학습

In [77]:
predictr.fit(X,y)

/home/cgb2/anaconda3/envs/ag/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)

## D. 예측

In [78]:
#predictr.predict(XX)

## E. 평가

In [79]:
predictr.score(X,y)

## G. 해석

In [80]:
pd.DataFrame({'X':predictr.feature_names_in_,'importance':predictr.feature_importances_})\
.sort_values('importance',ascending=False)

# 6. HW

위와 동일한 방식으로 `test.csv`에서의 생존여부를 예측하고 kaggle에 제출

In [81]:
submission.assign(Survived = predictr.predict(XX).astype(int)).to_csv("submission.csv",index=False)
!kaggle competitions submit -c titanic -f submission.csv -m '["Pclass", "Sex", "SibSp", "Parch"]'
!rm submission.csv

100%|██████████████████████████████████████| 2.77k/2.77k [00:02<00:00, 1.19kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster